In [ ]:
'''
Table: Products

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| product_id    | int     |
| new_price     | int     |
| change_date   | date    |
+---------------+---------+
(product_id, change_date) is the primary key (combination of columns with unique values) 
of this table.
Each row of this table indicates that the price of some product was changed 
to a new price at some date.
 

Write a solution to find the prices of all products on 2019-08-16. 
Assume the price of all products before any change is 10.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Products table:
+------------+-----------+-------------+
| product_id | new_price | change_date |
+------------+-----------+-------------+
| 1          | 20        | 2019-08-14  |
| 2          | 50        | 2019-08-14  |
| 1          | 30        | 2019-08-15  |
| 1          | 35        | 2019-08-16  |
| 2          | 65        | 2019-08-17  |
| 3          | 20        | 2019-08-18  |
+------------+-----------+-------------+
Output: 
+------------+-------+
| product_id | price |
+------------+-------+
| 2          | 50    |
| 1          | 35    |
| 3          | 10    |
+------------+-------+
'''

<pre>
# Write your MySQL query statement below

WITH base_table AS (
		SELECT *
			,rank() OVER (
				PARTITION BY product_id ORDER BY change_date DESC
				) AS ranking
		FROM Products
		WHERE change_date <= '2019-08-16'
		)
SELECT A.product_id
	,
IF (
		new_price IS NULL
		,10
		,new_price
		) AS price FROM (
	SELECT DISTINCT product_id
	FROM Products
	) A LEFT JOIN (
	SELECT *
	FROM base_table
	WHERE ranking = 1
	) B ON A.product_id = B.product_id
</pre>

## Another Approach

<pre>
# Write your MySQL query statement below

WITH on_date_items AS (
		SELECT product_id
			,new_price AS price
		FROM Products
		WHERE change_date = '2019-08-16'
		)
	,before_date_item AS (
		SELECT p.product_id
			,p.new_price AS price
		FROM Products p
		INNER JOIN (
			SELECT product_id
				,max(change_date) AS change_date
			FROM Products
			WHERE change_date < '2019-08-16'
				AND product_id NOT IN (
					SELECT product_id
					FROM on_date_items
					)
			GROUP BY product_id
			) x ON (
				x.product_id
				,x.change_date
				) = (
				p.product_id
				,p.change_date
				)
		)
	,after_date_items AS (
		SELECT p.product_id
			,10 AS price
		FROM Products p
		INNER JOIN (
			SELECT product_id
				,max(change_date) AS change_date
			FROM Products
			WHERE change_date > '2019-08-16'
				AND product_id NOT IN (
					SELECT product_id
					FROM on_date_items
					)
				AND product_id NOT IN (
					SELECT product_id
					FROM before_date_item
					)
			GROUP BY product_id
			) x ON (
				x.product_id
				,x.change_date
				) = (
				p.product_id
				,p.change_date
				)
		)
SELECT *
FROM on_date_items
UNION
SELECT *
FROM before_date_item
UNION
SELECT *
FROM after_date_items;
</pre>